## Introduction

As an improvement upon the weighted mean baseline ([notebook](https://www.kaggle.com/code/pankajpansari/baseline-0-weighted-mean-probabilities)), we're going to use a 2D CNN to make predictions.

A CT scan series is composed of many slices, that is, 2D images. Hence, each series is a sequence of 2D images. Let us ignore this sequential information, and think about making predictions on single 2D images. Two questions arise:

1. How do we make the training set?
    The labels are assigned to patients. Each patient can have multiple CT scans. Each CT scan series has many images. Let's do the simplest thing and assign to each image in each series, the label of the respective patient for all injury types.
    
2. Given predictions at image level, how do we combine them into predictions for patients?
    The first idea is to use the mean. However, it may be that only a few images in the CT scan indicate some type of injury and that may be enough for the injury to be present. We could use max to aggregrate, but to avoid the noise/inaccuracies in the predictions, let's use 75% quantile value.

So in this notebook, we are going to fine-tune a pretrained CNN model using the CT scan images from the competition. Specifically, we are going to take a ResNet-18 model pre-trained on ImageNet and fine tune it on our CT scan data. We'll make use of the [fastai](https://docs.fast.ai) library.

Here, we are not going to make use of segmentation data, DICOM tags, or meta data. To make our task simpler, we'll work with 128x128 images. We obtained this image data by taking the PNG dataset of [notebook](https://www.kaggle.com/code/theoviel/get-started-quicker-dicom-png-conversion) and resizing them beforehand using [this script](https://www.kaggle.com/code/pankajpansari/rsna-2023-abdominal-trauma-converting-dicom-to-png).

## Code

### EDA
We import the necessary modules.

In [ ]:
!pip install kornia pydicom

In [ ]:
import numpy as np
import pandas as pd
import os, random
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from fastai.vision.all import *
from fastai.basics import *
from fastai.callback.all import *
from fastai.medical.imaging import *
import shutil
import pydicom
import cv2
import glob
import time
import seaborn as sns
import torchvision
from sklearn.model_selection import train_test_split
from joblib import Parallel, delayed

from tqdm.notebook import tqdm
from joblib import Parallel, delayed

random.seed(42)

In [ ]:
BASE_DIR = '.'

Let us look at the labels provided in *train.csv*. We note that the labels are assigned to each patient and not to each CT scan image. There are many CT images associated with each patient; some of them may reflect the injury and some may not, depending upon their z-coordinate.

However, for simplification, for each CT scan image we'll simply assign the label given to the respective patient.

In [ ]:
patient_labels = pd.read_csv(os.path.join(BASE_DIR, 'train.csv'))
patient_labels.head()

We do a sanity check that the *healthy* and *injury* (or *low injury* and *high injury*) probabilities add up to 1 for each type of injury.

In [ ]:
#Check data has no NANs
print(patient_labels.isnull().any().any())
#Check healthy and injury labels are complementary
print((patient_labels['bowel_healthy'] == np.abs(1 - patient_labels['bowel_injury'])).all())
print((patient_labels['extravasation_healthy'] == np.abs(1 - patient_labels['extravasation_injury'])).all())
print((patient_labels['kidney_healthy'] == np.abs(1 - patient_labels['kidney_low'] - patient_labels['kidney_high'])).all())
print((patient_labels['liver_healthy'] == np.abs(1 - patient_labels['liver_low'] - patient_labels['liver_high'])).all())
print((patient_labels['spleen_healthy'] == np.abs(1 - patient_labels['spleen_low'] - patient_labels['spleen_high'])).all())

The *any_injury* label seems a bit redundant since it can be derived from the other injury labels. It simply means whether at least one type of bowel/extravsation/liver/spleen/kidney injury is present.

In [ ]:
#Check consistency of any_injury label with respect to other labels
print((patient_labels['any_injury'] == 1 - np.min(patient_labels[['bowel_healthy', 'extravasation_healthy', 
                                                          'kidney_healthy', 'liver_healthy', 'spleen_healthy']], axis = 1)).all())

#### Checking Data Imabalance and Correlations

Let us find the percentage of samples having different forms of injuries. This will help us know the distribution of the labels and find out whether there is imbalance in positive/negative samples.

In [ ]:
def get_pos_percent(df, label):
    num_entries = df.shape[0]
    return (df[label] == 1).sum()*100/num_entries

injury_labels = ['bowel_injury', 'extravasation_injury' , 'kidney_low' , 'kidney_high', 'liver_low',
                 'liver_high', 'spleen_low', 'spleen_high']


for label in injury_labels:    
    print(f'% of {label} samples = {get_pos_percent(patient_labels, label): .3f}')

Indeed there is a lot of imbalance between number of positive and negative samples for each type of injury.

Now, let us check how much the injuries are correlated among themselves. We'll plot a heatmap of the Pearson correlation coefficients (between -1 and 1).

In [ ]:
sns.heatmap(patient_labels[injury_labels].corr(), vmin = -1, vmax = 1, annot = True)

The correlations are very weak, with most values close to 0 and maximum as 0.2.

### Data Preparation

#### Splitting dataset

Let us first form training and validation sets. Because of the imbalance, we would like to ensure that the class/injury distribution is similar in the three sets.

In [ ]:
train_patients, validation_patients = train_test_split(patient_labels, test_size = 0.2, random_state = 42) 

In [ ]:
print('Number of samples')
print(f'Training set: {train_patients.shape[0]}   Validation set: {validation_patients.shape[0]}')

In [ ]:
print(f'% of positive samples - Training, Validation')
for label in injury_labels:    
    print(f'{label}: {get_pos_percent(train_patients, label): .3f}, {get_pos_percent(validation_patients, label): .3f}')

The distribution of the labels in not exactly equal in the training and validation sets, but it is more or less similar. For now, this will do for our purposes.

#### Multi-label Classification Data

We are going to consider our problem as multi-label classification. This is because a patient can have one of five different types of injuries, and the injuries can potentially co-exist.

We're going to use the patient labels in _train.csv_ to derive labels for each image.

In [ ]:
filename_labels = pd.DataFrame()
f_list = os.listdir('/storage/rsna-atd-128-png-pt1')
filename_labels['fname'] = pd.Series(f_list)

From _train.csv_ , we construct a dictionary _patient_dict_ which has patient_id as key and the list of injuries as labels.

In [ ]:
train_patients['is_valid'] = False
validation_patients['is_valid'] = True
train_val_df = pd.concat([train_patients, validation_patients])

In [ ]:
train_val_df['is_valid'].value_counts()

In [ ]:
patient_dict = {}

target_labels = ['bowel_injury', 'extravasation_injury' , 'kidney_healthy' , 'kidney_low' , 'kidney_high', 'liver_healthy', 'liver_low',
                 'liver_high', 'spleen_healthy', 'spleen_low', 'spleen_high']
reduced_target_labels = ['bowel_injury', 'extravasation_injury' , 'kidney_injury', 'liver_injury', 'spleen_injury']


for idx, patient_id in enumerate(train_val_df['patient_id']):
    entry = train_val_df.iloc[idx][target_labels]
    is_valid = train_val_df.iloc[idx]['is_valid'] 
    patient_dict[patient_id] = (entry, is_valid)

Having constructed our _patient_dict_ dictionary, we can loop through all the PNG filenames and for each look up the appropriate entry from the dictionary using _patient_id_ as the key. We encode bowel and extravasation injury as binary variable. We encode liver, kidney and spleen as 0/1/2 depending on whether there is no injury (healthy), low injury or high injury.

It was important to construct a dictionary first, so that this lookup can be fast using hash table.

In [ ]:
label_list = []
is_valid_list = []
reduced_target_labels = ['bowel_injury', 'extravasation_injury' , 'kidney_injury', 'liver_injury', 'spleen_injury']

start = time.time()
for scan_name in tqdm(filename_labels['fname']):
    patient_id = int(scan_name.split('_')[0])
    entry = patient_dict[patient_id][0]
    
    if_bowel = entry['bowel_injury']
    if_extravasation = entry['extravasation_injury']
    if_kidney = max(1*entry['kidney_healthy'], 2*entry['kidney_low'], 3*entry['kidney_high']) - 1
    if_liver = max(1*entry['liver_healthy'], 2*entry['liver_low'], 3*entry['liver_high']) - 1
    if_spleen = max(1*entry['spleen_healthy'], 2*entry['spleen_low'], 3*entry['spleen_high']) - 1

    labels = [if_bowel, if_extravasation, if_kidney, if_liver, if_spleen]
    label_list.append(labels)
    
    is_valid = patient_dict[patient_id][1]
    is_valid_list.append(is_valid)

end = time.time()
print(end - start)

In [ ]:
filename_labels.shape

In [ ]:
filename_labels[reduced_target_labels] = pd.DataFrame(label_list)
filename_labels['is_valid'] = pd.Series(is_valid_list)

In [ ]:
assert(filename_labels['bowel_injury'].all() in [0, 1])
assert(filename_labels['extravasation_injury'].all() in [0, 1])
assert(filename_labels['kidney_injury'].all() in [0, 1, 2])
assert(filename_labels['liver_injury'].all() in [0, 1, 2])
assert(filename_labels['spleen_injury'].all() in [0, 1, 2])

We have the data in the format we wanted. Now we can construct the dataloader and train a CNN model.

In [ ]:
print(filename_labels.shape)
filename_labels.head()

### Training

We're going to take a ResNet-18 model pretrained on ImageNet (directly available via fastai library) and fine-tune it on our small data.

First, we construct our dataloader. Here, we do not perform any data augmentation.

In [ ]:
SIZE = 128

bowel_vocab = filename_labels['bowel_injury'].unique()
extravasation_vocab = filename_labels['extravasation_injury'].unique()
kidney_vocab = filename_labels['kidney_injury'].unique()
liver_vocab = filename_labels['liver_injury'].unique()
spleen_vocab = filename_labels['spleen_injury'].unique()

blocks = (ImageBlock(cls=PILImageBW), 
          CategoryBlock(vocab = bowel_vocab),
          CategoryBlock(vocab = extravasation_vocab),
          CategoryBlock(vocab = kidney_vocab),
          CategoryBlock(vocab = liver_vocab),
          CategoryBlock(vocab = spleen_vocab))

In [ ]:
kidney_vocab

In [ ]:
getters = (ColReader('fname', pref = '/storage/rsna-atd-128-png-pt1/'), 
           ColReader('bowel_injury'), ColReader('extravasation_injury'),
           ColReader('kidney_injury'), ColReader('liver_injury'), ColReader('spleen_injury'))

In [ ]:
data_block = DataBlock(blocks = blocks,
                       getters = getters,
                       splitter = ColSplitter('is_valid'),
                       n_inp = 1)

In [ ]:
dls = data_block.dataloaders(filename_labels, bs = 128)

In [ ]:
dls.c

Let  us visualize one batch. The labels are of the form (y1, y2, y3, y4, y5) where y1 represents bowel_injury and can take 0 or 1. y5 represents spleen_injury and can take 0/1/2.

In [ ]:
dls.show_batch()

We need to change the head of the ResNet-18 model because we're dealing with a multi-label problem instead of a multi-class problem (for which the network was pretrained on ImageNet. Essentially, we're going to remove the output layer and replace it by one output node for bowel (binary), one for extravasation (binary), three for liver (ternary), three for kidney, and three for spleen. Hence, we have 5 new heads which we are going to connect to the *body* of ResNet18 - body is the full-network minus the layer output layer. 

In [ ]:
class MultiHeadModel(Module):
    
    def __init__(self, body):
    
        self.body = body
        nf = num_features_model(nn.Sequential(*self.body.children()))

        self.bowel = create_head(nf, 1)
        self.extravasation = create_head(nf, 1)
        self.kidney = create_head(nf, 3)
        self.liver = create_head(nf, 3)
        self.spleen = create_head(nf, 3)
        
    def forward(self, x):
        
        y = self.body(x)
        bowel = self.bowel(y)
        extravasation = self.extravasation(y)
        kidney = self.kidney(y)
        liver = self.liver(y)
        spleen = self.spleen(y)
        return [bowel, extravasation, kidney, liver, spleen]

In [ ]:
base_model = create_vision_model(models.resnet18, 10, True, n_in = 1)
body = create_body(base_model, pretrained=True)
net = MultiHeadModel(body)

Our loss is now combination of binary cross-entropy losses for the bowel and extravasation labels, and multi-class cross-entropy loss for kidney, liver, and spleen. Given the sample weights in the competition, we introduce weights for each loss type; hence mistakes in some categories (such as extravasation) are penalized much more than others (such as bowel). For ternary categories, the weight is the average of the weights of the low and high levels ( (2+4)/2 = 3).

In [ ]:
class CombinationLoss(Module):
    "Cross entropy loss on multiple targets"
    def __init__(self, weights = [2, 6, 3, 3, 3]):
        self.w = weights
        
    def forward(self, xs, *ys, reduction = 'mean'):
        loss = 0
        for i, w, x, y in zip(range(len(xs)), self.w, xs, ys):
            if i < 2:
                #loss += w*F.binary_cross_entropy_with_logits(x, y.unsqueeze(1).float(), reduction = reduction)
                loss += w*torchvision.ops.sigmoid_focal_loss(x, y.unsqueeze(1).float(), reduction = reduction)
            else:
                loss += w*F.cross_entropy(x, y, reduction = reduction)
        return loss

Because we observed that there is severe imabalance in the data, accuracy is not a good metric to judge the quality of our model. Instead, we are going to look at the per-category recall and the average recall.

In [ ]:
from sklearn.metrics import recall_score

class RecallPartial(Metric):
    "Stores predictions and targets on CPU in accumulate to perform final calculations with `func`."
    def __init__(self, a=0, **kwargs):
        self.func = partial(recall_score, average='macro', zero_division=0)
        self.a = a

    def reset(self): self.targs,self.preds = [],[]

    def accumulate(self, learn):
        if self.a < 2:
            pred = learn.pred[self.a]>0
        else:
            pred = learn.pred[self.a].argmax(-1)
        targ = learn.y[self.a]
        pred,targ = to_detach(pred),to_detach(targ)
        pred,targ = flatten_check(pred,targ)
        self.preds.append(pred)
        self.targs.append(targ)

    @property
    def value(self):
        if len(self.preds) == 0: return
        preds,targs = torch.cat(self.preds),torch.cat(self.targs)
        return self.func(targs, preds)

    @property
    def name(self): return 'recall_' + filename_labels.columns[self.a+1].split('_')[0]
    
class RecallCombine(Metric):
    
    def accumulate(self, learn):
        scores = [learn.metrics[i].value for i in range(3)]
        self.combine = np.average(scores, weights=[2,1,1])

    @property
    def value(self):
        return self.combine

We finally create a fast.ai learner object that encapsulates the dataset, model, loss function, and metrics.

In [ ]:
learn = Learner(dls, net, loss_func = CombinationLoss(), metrics=[RecallPartial(a=i) for i in range(len(dls.c))] + [RecallCombine()],
               default_cbs = False, cbs = [TrainEvalCallback, Recorder(train_metrics = False, valid_metrics = True), ProgressCallback])

Let's look at the default optimization settings used by fast.ai

In [ ]:
learn.opt_func

We do mixed-precision training to speed up our training process.

In [ ]:
learn.to_fp16()

fastai provides a handy function to let us find a good learning rate.

In [ ]:
learn.lr_find()

Let's now fine tune our model for 4 epochs using the recommended learning rate.

In [ ]:
learn.fine_tune(4, 1.5e-3)

As the epochs progress, the training error decreases to close to 0 (perfect recall for all categories for all images in the training set). However, the validation error is large and does not improve. The recall metrics on the validation set also indicate not very good performance.

Perhaps our idea of assigning the same labels to all images in the CT scan series is not a good one. For instance, any organ is only present in a subset of images and absent in others. Besides, we've discarded the temporal information. Because of these, there is no meaningful pattern to learn and the network simply overfits to the training data.

### Save Model

We need to save our model, so that our [inference notebook](https://www.kaggle.com/code/pankajpansari/rsna-2023-atd-baseline-1-inference) can import it to make predictions on the test set.

In [ ]:
learn.export('model_2.pt')

In [ ]:
a_dict = {'dataset': 'rsna-atd-128-png-pt1',
     'model_name': 'model_2',
     'architecture': 'resnet18',
     'bs': 128,
     'opt': 'adam with fastai defaults',
     'lr': 1.5e-3,
     'epochs': 4
}

In [ ]:
import json
with open('saved_models.json', 'a') as outfile:
    json.dump(a_dict, outfile)

In [ ]:
#import gc; gc.collect()
#torch.cuda.empty_cache()

In [ ]:
#!watch -n 1 nvidia-smi
#!nvidia-smi --query-gpu=timestamp,pstate,temperature.gpu,utilization.gpu,utilization.memory,memory.total,memory.free,memory.used --format=csv -l 1